# US Bureau of Labor Statistics Occupational Employment and Wage Estimates by Metropolitan and Nonmetropolitan Area Data Cleaner

The datasets are seperated by year, with each year having multiple `.xls` files where the different areas are split up. This notebook aims to clean the data from 2006-2024 by removing unwanted areas, and generating datasets separated by years and an overall dataset.

We take data starting from 2006 since that is the earliest data by both metropolitan and nonmetropolitan area available, and we want to analyze the time range at which the job market affects college major choice. This enables us to plot up to the relationship between the 2006 job market and 2014 graduation majors (an 8 year difference).

In [2]:
# Import necessary libraries

import os
import pandas as pd

In [ ]:
# Changes all xls files into csv files

def xls_x_to_csv(file_path, file_type):
    # Read the .xls file
    df = pd.read_excel(file_path)
    # Define the .csv file path
    csv_file_path = file_path.replace(file_type, '.csv')
    # Save the dataframe to a .csv file
    df.to_csv(csv_file_path, index=False)

def all_xls_x_to_csv(bls_data_path):
    # Iterate over each year folder in the bls_data folder
    for year_folder in os.listdir(bls_data_path):
        if year_folder == "full_processed_csvs" or year_folder == "useful_csvs":
            continue
        year_path = os.path.join(bls_data_path, year_folder)
        if os.path.isdir(year_path):
            # Iterate over each .xls file in the year folder
            for file in os.listdir(year_path):
                if file.endswith('.xls'):
                    file_path = os.path.join(year_path, file)
                    xls_x_to_csv(file_path, '.xls')
                if file.endswith('.xlsx'):
                    file_path = os.path.join(year_path, file)
                    xls_x_to_csv(file_path, '.xlsx')
                    
# Uncomment the line below to change all xls and xlsx files into csv files
# all_xls_x_to_csv('bls_data')

In [23]:
# Read through each year of csvs, and get only relevant areas into one csv
relevant_areas = ["Los Angeles-Long Beach-Santa Ana, CA", 
"Los Angeles-Long Beach-Anaheim, CA", # shifts classification in 2015
"Western New Hampshire nonmetropolitan area",
"Southwestern New Hampshire nonmetropolitan area",
"West Central New Hampshire nonmetropolitan area",
"Southwest New Hampshire nonmetropolitan area",
"West Central-Southwest New Hampshire nonmetropolitan area",
"San Diego-Carlsbad-San Marcos, CA",
"San Diego-Carlsbad, CA", # shifts classification in 2015
"Ann Arbor, MI",
"Washington-Arlington-Alexandria, DC-VA-MD-WV Metropolitan Division",
"Spokane, WA",
"Spokane-Spokane Valley, WA", # shifts classification in 2015
"Winston-Salem, NC",
"Utica-Rome, NY",
]

# All the relevant New Hampshire areas:
# "Other New Hampshire nonmetropolitan area", - this means central - do not use
# "Western New Hampshire nonmetropolitan area", - use
# "Southwestern New Hampshire nonmetropolitan area", - use
# "West Central New Hampshire nonmetropolitan area", - use - 2015
# "Central New Hampshire nonmetropolitan area", - do not use - 2015
# "Southwest New Hampshire nonmetropolitan area", - use - 2015
# "West Central-Southwest New Hampshire nonmetropolitan area", - 2018 combination of west central and southwest - only choice aside from central in 2018 onwards
# overall, we'll use West Central-Southwest, while avoiding Central?

def process_year(bls_data_path, year_path, relevant_areas, column_name, year_number):
    # Create the output directory if it doesn't exist
    processed_folder = os.path.join(bls_data_path, "full_processed_csvs")
    os.makedirs(processed_folder, exist_ok=True)

    result_df = pd.DataFrame()

    # Iterate through all CSV files in the year folder
    for file in os.listdir(year_path):
        if file.endswith('.csv'):
            file_path = os.path.join(year_path, file)
            df = pd.read_csv(file_path)
            if column_name in df.columns:
                # Filter rows where the column value is in relevant_areas
                filtered_df = df[df[column_name].isin(relevant_areas)]
                result_df = pd.concat([result_df, filtered_df], ignore_index=True)

    # Define the output file path
    output_file_path = os.path.join(processed_folder, f"{year_number}_relevant.csv")

    # Write the filtered DataFrame to the output file
    result_df.to_csv(output_file_path, index=False)
    print(f"Processed data saved to {output_file_path}")

def process_all_years(bls_data_path, relevant_areas):
    # Iterate over each year folder in the bls_data folder
    for year_folder in os.listdir(bls_data_path):
        if year_folder == "full_processed_csvs" or year_folder == "useful_csvs":
            continue
        year_number = int(year_folder)
        year_path = os.path.join(bls_data_path, year_folder)
        column_name = ''
        if os.path.isdir(year_path):
             # AREA_NAME 2006-2018, area_title in 2019, AREA_TITLE onwards
            if year_number < 2019:
                column_name = 'AREA_NAME'
            elif year_number == 2019:
                column_name = 'area_title'
            elif year_number > 2019:
                column_name = 'AREA_TITLE'
            else:
                print("Error: invalid year")
                exit(1)
            process_year(bls_data_path, year_path, relevant_areas, column_name, year_number)

process_all_years('bls_data', relevant_areas)

Processed data saved to bls_data/full_processed_csvs/2013_relevant.csv
Processed data saved to bls_data/full_processed_csvs/2014_relevant.csv
Processed data saved to bls_data/full_processed_csvs/2022_relevant.csv
Processed data saved to bls_data/full_processed_csvs/2023_relevant.csv
Processed data saved to bls_data/full_processed_csvs/2015_relevant.csv
Processed data saved to bls_data/full_processed_csvs/2012_relevant.csv
Processed data saved to bls_data/full_processed_csvs/2008_relevant.csv
Processed data saved to bls_data/full_processed_csvs/2006_relevant.csv
Processed data saved to bls_data/full_processed_csvs/2007_relevant.csv
Processed data saved to bls_data/full_processed_csvs/2009_relevant.csv
Processed data saved to bls_data/full_processed_csvs/2017_relevant.csv
Processed data saved to bls_data/full_processed_csvs/2010_relevant.csv
Processed data saved to bls_data/full_processed_csvs/2019_relevant.csv
Processed data saved to bls_data/full_processed_csvs/2021_relevant.csv
Proces

In [24]:
# Format all area names and column names to be consistent, more cleaning done in next step

def format_area_names(df):
    if 'AREA_NAME' in df.columns:
        df.rename(columns={'AREA_NAME': 'AREA_TITLE'}, inplace=True)
    df['AREA_TITLE'] = df['AREA_TITLE'].replace("Los Angeles-Long Beach-Santa Ana, CA", "Los Angeles-Long Beach-Anaheim, CA")
    df['AREA_TITLE'] = df['AREA_TITLE'].replace("San Diego-Carlsbad-San Marcos, CA", "San Diego-Carlsbad, CA")
    df['AREA_TITLE'] = df['AREA_TITLE'].replace("Spokane, WA", "Spokane-Spokane Valley, WA")
    df['AREA_TITLE'] = df['AREA_TITLE'].replace("Western New Hampshire nonmetropolitan area", "West Central-Southwest New Hampshire nonmetropolitan area")
    df['AREA_TITLE'] = df['AREA_TITLE'].replace("Southwestern New Hampshire nonmetropolitan area", "West Central-Southwest New Hampshire nonmetropolitan area")
    df['AREA_TITLE'] = df['AREA_TITLE'].replace("West Central New Hampshire nonmetropolitan area", "West Central-Southwest New Hampshire nonmetropolitan area")
    df['AREA_TITLE'] = df['AREA_TITLE'].replace("Southwest New Hampshire nonmetropolitan area", "West Central-Southwest New Hampshire nonmetropolitan area")
    if 'GROUP' in df.columns:
        df.rename(columns={'GROUP': 'OCC_GROUP'}, inplace=True)
    elif 'O_GROUP' in df.columns:
        df.rename(columns={'O_GROUP': 'OCC_GROUP'}, inplace=True)
    if 'LOC QUOTIENT' in df.columns:
        df.rename(columns={'LOC QUOTIENT': 'LOC_QUOTIENT'}, inplace=True)
    if 'LOC_QUOTIENT' not in df.columns:
        df['LOC_QUOTIENT'] = pd.NA
    
    return df

def format_column_names(df):
    for column in df.columns:
        df.rename(columns={column: column.upper()}, inplace=True)
    return df

def format_year(file_path):
    df = pd.read_csv(file_path)
    df = format_column_names(df)
    df = format_area_names(df)
    df.to_csv(file_path, index=False)

def format_all_years(processed_csv_path):
    for file in os.listdir(processed_csv_path):
        if file.endswith('.csv'):
            file_path = os.path.join(processed_csv_path, file)
            format_year(file_path)

format_all_years('bls_data/full_processed_csvs')

In [32]:
# Finish cleaning data by dropping useless columns

def drop_useless_columns(df):
    # Drop columns that are not needed
    drop_list = ['AREA_TYPE', 'NAICS', 'NAICS_TITLE', 'I_GROUP', 'OWN_CODE', 'OCC_CODE', 'PCT_TOTAL', 'PCT_RPT', 'PRIM_STATE', 'AREA']
    # OCC_CODE is the same as OCC_TITLE, PCT_TOTAL and PCT_RPT is always NaN, PRIM_STATE is in AREA_TITLE, AREA is same as AREA_TITLE, others cause issues
    for column in drop_list:
        if column in df.columns:
            df.drop(columns=[column], inplace=True)
    df.dropna(axis=1, how='all')
    return df

def drop_all_useless_columns(processed_csv_path):
    for file in os.listdir(processed_csv_path):
        if file.endswith('.csv'):
            file_path = os.path.join(processed_csv_path, file)
            df = pd.read_csv(file_path)
            out_df = drop_useless_columns(df)
            out_df.to_csv(os.path.join('bls_data/useful_csvs', file), index=False)

drop_all_useless_columns('bls_data/full_processed_csvs')

In [ ]:
# Test to check if all years have the same columns and if all areas are represented

def test_csvs(useful_csv_path):
    reference_columns = None
    reference_area_titles = None
    for file in os.listdir(useful_csv_path):
        if file.endswith('.csv'):
            file_path = os.path.join(useful_csv_path, file)
            df = pd.read_csv(file_path)
            if reference_columns is None:
                reference_columns = set(df.columns)
            else:
                if set(df.columns) != reference_columns:
                    print(f"Warning: Columns in {file} do not match the reference columns.")
            if reference_area_titles is None:
                reference_area_titles = set(df['AREA_TITLE'].unique())
            else:
                if set(df['AREA_TITLE'].unique()) != reference_area_titles:
                    print(f"Warning: Areas in {file} do not match the reference areas.")
            unique_area_titles = df['AREA_TITLE'].nunique()
            if unique_area_titles != 8:
                print(f"Warning: {file} does not have all 8 areas represented.")
            

In [34]:
# Combine all years into one csv

def combine_years(useful_csv_path):
    combined_df = pd.DataFrame()
    for file in os.listdir(useful_csv_path):
        if file.endswith('.csv') and file != 'all_years.csv':
            file_path = os.path.join(useful_csv_path, file)
            df = pd.read_csv(file_path)
            year = int(file.split('_')[0])
            df['YEAR'] = year
            combined_df = pd.concat([combined_df, df], ignore_index=True)
    combined_df.dropna(axis=1, how='all')
    column_order = ['YEAR', 'AREA_TITLE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP', 'EMP_PRSE', 'JOBS_1000', 'LOC_QUOTIENT', 'H_MEAN', 'A_MEAN', 'MEAN_PRSE', 
    'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90', 'A_PCT10', 'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'ANNUAL', 'HOURLY']
    combined_df = combined_df[column_order]
    combined_df = combined_df.sort_values(by=['YEAR', 'AREA_TITLE'], ascending=[True, True], kind='mergesort')
    combined_df.to_csv('bls_data/useful_csvs/all_years.csv', index=False)

combine_years('bls_data/useful_csvs')

In [4]:
df = pd.read_csv('bls_data/useful_csvs/all_years.csv')
df.shape

(74580, 23)